In [29]:
import json
import numpy as np
from tqdm.auto import trange, tqdm
import re
import pandas as pd


def tn_to_arb(line):
  fline = ''
  for char in line:
    if ord(char) >= ord('๐') and ord(char) <= ord('๙'):
      fline += str(ord(char) - ord('๐'))
    else:
      fline += char
  return fline

def get_level_of_line(String):
  regx = [('\d+(\.\d+){2}\)', 10),
   ('\d+\.\d+\)', 9),
   ('\d+\)', 8),
   ('กิจกรรม', 7),
   ('\(\d*(\.?\d*)*\)', 12),   # (x), (x.x), (x.x.x)
   ('\d+(\.\d+){5}\D+', 6),
   ('\d+(\.\d+){4}\D+', 5),
   ('\d+(\.\d+){3}\D+', 4),
   ('\d+(\.\d+){2}\D+', 3),
   ('\d+\.\d+\D+', 2),
   ('\d+\.\D+', 1),
   ('\d+.', 11)]

  for r, l in regx:
    if re.match(r, String):
      return l
  return 0

def get_patern_of_bullet(String):
  String += ' '
  regx = [('[1-9](\.[1-9])*\)', 20),
   ('กิจกรรม', 40),
   ('\(\d*(\.?\d*)*\)', 50),
   ('[1-9](\.[1-9])+', 2),
   ('[1-9]\.[^0]', 1),
   ('[1-9][^.]', 30)]

  for r, l in regx:
    if re.match(r, String):
      if l in [2, 9, 12]:
        l = String.count('.') + 1
      return r, l
  return 0

def mode(lst):
  if isinstance(lst, np.ndarray):
    lst = lst.tolist()
  return max(set(lst), key=lst.count)

In [3]:
def to_gregorian(input):
  if input:
    return int(input) - 543
  else:
    return input

In [4]:
class ThaiBudget:
  def __init__(self, json_path):
    with open(json_path) as json_file:
      self._data_ = json.load(json_file)
      self.docs = [ref_doc for ref_doc in self._data_]
  
  def get_doc(self, ref_doc):
    return Doc(self._data_[ref_doc], ref_doc)


class Doc:
  def page(self, num_page, true_num_page=False):
    if true_num_page:
      npage = num_page + self.start
      return Page(npage, self._raw_page[npage], num_page)
    else:
      return Page(num_page, self._raw_page[num_page])

  @property
  def index_page(self):
    id_pages = []
    for i in range(30):
      page = self.page(i).get_text_lines()
      if page != None:
        text = ''.join(page)
        if re.findall('\d+ (ถึง) \d+', text):
          id_pages.append(i)
        elif id_pages:
          break
    return id_pages

  @property
  def ministry(self):
    ministry = []
    lines = self.page(0)._lines(yPos=True)
    for tline, ypline in lines:
      if (np.array([1250, 1400]) < ypline).any() and (ypline < np.array([2000, 2100])).any():
        ministry.append(' '.join([tn_to_arb(block) for block in tline if block not in '*-']))
    return ministry

  @property
  def range(self):
    page_rage = []
    ministry = ''
    tempdict = {}
    stw = [ministr[:int(len(ministr)*0.6)]for ministr in self.ministry]
    for i in self.index_page:
      lines = self.page(i).get_text_list_lines()
      for line in lines:
        if [sw for sw in stw if ' '.join(line).startswith(sw)]:
          ministry = [word for word in line if word != 'ถึง' and not word.isdigit() and word != 'หน้า']
          ministry = ' '.join(ministry)
        if re.match('\(\d+\)', line[0]):
          if tempdict:
            page_rage.append(tempdict)
            tempdict = {}
          tempdict['ministry'] = ministry
          tempdict['budget_unit'] = ' '.join([word for word in line[1:] if word != 'ถึง' and not word.isdigit()])
        if '7. ราย' in ' '.join(line):
          start = line[-1]
        if '8. ราย' in ' '.join(line):
          tempdict['range'] = (int(start), int(line[-1]))
    return page_rage

  @property
  def start(self):
    idpage = self.index_page
    fstpage = idpage[-1] + 1
    while self.page(fstpage).get_blocks() == None:
      fstpage += 1
    return fstpage - 1
  
  def get_budgetary_unit(self, minis):
    budumi = []
    for budu in self.range:
      if minis == budu['ministry']:
        budumi.append({'budget_unit': budu['budget_unit'],
                      'range': budu['range']})
      
    return budumi

  def get_ministry(self, budge_u):
    for budu in self.range:
      if budge_u == budu['budget_unit']:
        return {'ministry': budu['ministry'],
                      'range': budu['range']}
      else:
        return None

  def __init__(self, raw_pages, ref_doc):
    self.fiscal_year, self.chabab, self.lem = ref_doc.split('.')
    self._raw_page = raw_pages
    self.ref_doc = ref_doc
    self.len = len(raw_pages)

class Page:
  def get_blocks(self):
    if self._page == []:
      return None
    blocks = []
    for block in self._page:
      crd = self._simplify_coordinates([[coo['x'], coo['y']] for coo in block['vertices']])
      blocks.append({'text': block['description'],
                    'sim_coord': crd})
    return blocks

  @property
  def xSigPos(self):
    w = self.whpage[0][1] - self.whpage[0][0]
    for line in self.Lines:
      text = [block['text'] for block in line]
      if [txt for txt in text if txt in ['ตั้งงบประมาณ', 'ผูกพันงบประมาณ', 'วงเงินทั้งสิ้น', 'ผลผลิต']]:
        continue
      scord = [block['sim_coord'] for block in line]
      for scrd, txt in zip(scord, text):
        if txt == 'บาท' and (scrd[0] > (self.whpage[0][1] - 200)).all():
          return scrd[0].tolist()

  @property
  def level_dict(self):
    lvldict = {}
    for line in self.Lines[1:]:
      fidx = line[0]
      pattern_lvel = get_patern_of_bullet(fidx['text'] + ' ')
      if pattern_lvel:
        if pattern_lvel[1] in lvldict.keys():
          lvldict[pattern_lvel[1]].append(fidx['sim_coord'][0][0])
        else:
          lvldict.update({pattern_lvel[1]: [fidx['sim_coord'][0][0]]})

    return {key: mode(lvldict[key]) for key in lvldict}

  def _simplify_coordinates(self, coord):
    if self._page == []:
      return None
    scrd = np.sort(coord, axis=0)
    x = [i[0] for i in scrd]
    y = [i[1] for i in scrd]
    xy = np.array([x[:2],x[2:], y[:2], y[2:]]).sum(axis=1)//2
    return np.array([xy[:2], xy[2:]])
  
  def _ypos_mean(self, list_of_blocks):
    if self._page == []:
      return None
    return np.mean([bx['sim_coord'][1] for bx in list_of_blocks], axis=0)

  def _lines(self, line_tolerance=None, yPos=False):
    line_tolerance = self.dline_tolerance if line_tolerance == None else line_tolerance
    if self._page == []:
      return None
    lines = []
    for b in self.get_blocks():
      if lines == []:
        lines.append([b])
      else:
        cm = b['sim_coord'][1]
        found = 0
        for line in lines:
          diff = abs(cm - self._ypos_mean(line))
          if diff[1] < line_tolerance:
            found = 1
            line.append(b)
            break
        if not found:
          lines.append([b])
    lines = [sorted(line, key = lambda i: i['sim_coord'][0][0]) for line in lines]
    if not yPos:
      return lines
    else:
      return [([block['text'] for block in line], self._ypos_mean(line).tolist()) for line in lines]

  def get_text_lines(self):
    if self._page == []:
      return []
    text_lines = []
    for line in self.get_text_list_lines():
      text_lines.append(' '.join(line))
    return text_lines

  def get_text_list_lines(self):
    if self._page == []:
      return []
    text_lines = []
    for line in self.Lines:
      text_lines.append([l['text'] for l in line])
    return text_lines    

  def xpos_text_lines(self):
    if self._page == []:
      return None
    xpos_lines = []
    text_lines = []
    for line in self.Lines:
      ax = np.array([bx['sim_coord'] for bx in line])
      xpos_lines.append(ax)
      text_lines.append([bx['text'] for bx in line])
    return  xpos_lines, text_lines

  def __init__(self, index_page, page, pdfpage=None):
    self.dline_tolerance = 30
    self.index_page = index_page
    self.pdfpage = pdfpage
    self._hpage = page[:1]
    self._page = page[1:]
    if page:
      sim = self._simplify_coordinates([[coo['x'], coo['y']] for coo in page[0]['vertices']])
    else:
      sim = [np.NaN] * 2
    self.whpage = sim
    self.Lines = self._lines()

In [5]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

# FUNCTIONS

In [6]:
cols = ['ITEM_ID','REF_DOC','REF_PAGE_NO','MINISTRY','BUDGETARY_UNIT',
        'CROSS_FUNC?','BUDGET_PLAN','OUTPUT','PROJECT','CATEGORY_LV1',
        'CATEGORY_LV2','CATEGORY_LV3','CATEGORY_LV4','CATEGORY_LV5',
        'CATEGORY_LV6','ITEM_DESCRIPTION','FISCAL_YEAR','AMOUNT','OBLIGED?', 'DEBUG_LOG']

In [59]:
def labelling(coord, text, whpage):
  labels = []
  xpos = []
  xbltpos = 0
  bltx = 0

  fscy = [txt for txt in text if txt in ['ตั้งงบประมาณ', 'ผูกพันงบประมาณ', 'ตั้งงบประมา']]

  jtxt = ''.join([txt for txt in text])
  skptw = ['เงินนอกงบประมาณ', 'เงินงบประมาณ', 'รายการบุคลากรภาครัฐ', 'วงเงินทั้งสิ้น', 'วงเงินทั้งสิ้น', 'วงเงินทั้งส้น',
           'วงเงินทิ้งส้น','วงเงินทั่้งสิ้น', 'วงเงินทิ่้งสิ้น', 'รายละเอียดงบประมาณจำแนกตามงบรายจ่าย']

  if [txt for txt in skptw if jtxt.startswith(txt)] or 'รายการผูกพัน' == jtxt:
    return None

  for crd, txt in zip(coord, text):
    xpos += crd[0].tolist()

    if (crd[1] < whpage[1][0] + 25).all() and ''.join(text).isdigit():
      labels.append('PAGENUM')
    elif (np.array([len(txt) for txt in text]) < 5).all():
        return None
    elif get_patern_of_bullet(jtxt) and not labels and not fscy:
      labels.append('BULLET')
      xbltpos = crd[0].tolist()[0]
      bltx = [crd[0]]
    elif (crd[1] < whpage[1][0] + 220).all() and (crd[0] < whpage[0][0] + 200).all() and txt == 'ผลผลิต':
      labels.append('OUTPUT')
      bltx = [crd[0]]
    elif (crd[1] < whpage[1][0] + 220).all() and (crd[0] < whpage[0][0] + 200).all() and txt == 'โครงการ':
      labels.append('PROJECT')
      bltx = [crd[0]]
    elif (crd[0] > whpage[0][1] - 250).any() and not fscy:
      if re.match(r'[0-9]{1,3}(,[0-9]{1,3})*', txt) and txt.replace(',','').isdigit():
        labels.append('AMOUNT')
      else:
        labels.append('BAHT')
    elif fscy:
      if txt.isdigit():
        labels.append('FYEAR')
      elif re.match(r'[0-9]{1,3}(,[0-9]{1,3})*', txt):
        labels.append('FAMOUNT')
      else:
        labels.append('FBAHT')
    else:
      if bltx and len(bltx) < 2:
        bltx.append(crd[0])
      labels.append('NOTMATCH')

  return xbltpos if xbltpos else np.min(xpos), text, labels, bltx


In [8]:
def clean_prefix(textlev, labellev):
  otext = []
  olabel = []
  for txt, lbel in zip(textlev, labellev):
    txtlbel = [{'text': text, 'label': label} for text, label in zip(txt, lbel)]
    if 'BULLET' in lbel:
      bltindex = lbel.index('BULLET')
      txt = txt[bltindex:]
      lbel = lbel[bltindex:]
    if 'FBAHT' in lbel :
      if 'ปี' in txt and 'FBAHT' in lbel:
        fyindex = txt.index('ปี')
        txt = txt[fyindex:]
        lbel = lbel[fyindex:]
    otext.append(txt)
    olabel.append(lbel)
  return otext, olabel

In [9]:
def get_budget_plan(plist):
  npagelist, poslist, lineslist = [], [], []
  for ipge in plist:
    tmppos, tmptext = ipge.xpos_text_lines()
    poslist += tmppos
    lineslist += tmptext
    npagelist += [ipge.pdfpage] * len(tmptext)

  budget_plan = ''
  for coord, line in zip(poslist, lineslist):
    if [block for block in line if re.match('7\.[1-9]\.[1-9]', block)]:
      return budget_plan
    elif [block for block in line if block.startswith('แผนงาน')]:
      for bcrd, btext in zip(coord, line):
        if (bcrd[0] < 1200).all() and not re.match('7\.[1-9]', btext):
          budget_plan += btext
    elif budget_plan:
      budget_plan += ' '.join(line)

In [10]:
def xpos_text_lines(Lines):
  xpos_lines = []
  text_lines = []
  for line in Lines:
    ax = [bx['sim_coord'] for bx in line]
    xpos_lines.append(ax)
    text_lines.append([bx['text'] for bx in line])
  return  xpos_lines, text_lines

In [11]:
def clean_entry(plist, pl=False):
  pstart = curr_doc.start

  if len(plist) > 1:
    allxpos = [ipage.xSigPos[0] for ipage in plist] 
    xdiffl = np.subtract(allxpos[0], allxpos)
  else:
    xdiffl = [0]
    
  npagelist, poslist, textlist, whplist = [], [], [], []

  for ipge, xdiff in zip(plist, xdiffl):
      tmppos, tmptext = xpos_text_lines(ipge.Lines)
      poslist += [np.add(tmpos, [[xdiff, xdiff], [0, 0]]) for tmpos in tmppos]
      textlist += tmptext
      npagelist += [ipge.pdfpage] * len(tmptext)
      whplist += [ipge.whpage] * len(tmptext)

  xposlev = []
  textlev = []
  labellev = []
  npagelev = []
  bltxlev = []

  for npage, coord, text, whp in zip(npagelist, poslist, textlist,whplist):
    olabel = labelling(coord, text, whp)

    if olabel == None:
      continue

    minxpos, textin, labels, bltx = olabel
    if 'PAGENUM' in labels:
      continue
    else:
      xposlev.append(minxpos)
      textlev.append(text)
      labellev.append(labels)
      npagelev.append(npage)
      bltxlev.append(bltx)

  textlev, labellev = clean_prefix(textlev, labellev)

  bltwdt = 0
  aftbltwdt = 0
  tbltx = 0
  oxposlev, otextlev, olabellev, onpagelev, obfbltxlev = [], [], [], [], []
  for idx, xpos, text, label, npage, bltx in zip(range(len(xposlev)), xposlev, textlev, labellev, npagelev, bltxlev):
    if isinstance(bltx, list):
      if len(bltx) < 2:
        print(bltx, text, label, npage)
      bltwdt = bltx[0][0] - bltx[0][1]
      aftbltwdt = bltx[0][0] - bltx[1][0]
      tbltx = bltx
    if oxposlev:
      if (('FBAHT' in label or 'FAMOUNT' in label or 'FYEAR' in label) or
          (('BAHT' in olabellev[-1] or 'FBAHT' in olabellev[-1]) and ('BULLET' in label or 'BAHT' in label)) or
          ('BULLET' in olabellev[-1] and 'BULLET' in label) or
          (len(labellev) > idx+1 and idx-1 >= 0 and 'BAHT' in labellev[idx-1] and 'BAHT' in labellev[idx+1] and 'BULLET' not in labellev[idx+1])):
        oxposlev.append(xpos)
        otextlev.append(text)
        olabellev.append(label)
        onpagelev.append(npage)
        obfbltxlev.append(tbltx)
      else:
        if ('BULLET' in olabellev[-1] and 'BULLET' in label):
          print('curr', xpos, text, label, npage)
        if isinstance(obfbltxlev[-1], int):
          print(' - ', xpos, text, label, npage)
        difblt = obfbltxlev[-1][0][0] - xpos
        if (difblt < 10 and not ('BULLET' in olabellev[-1] and 'BULLET' in label)  and (bltwdt - 15 < difblt or aftbltwdt - 20 < difblt or
                            (('OUTPUT' in olabellev[-1] and -113 < difblt) or 
                             ('PROJECT' in olabellev[-1] and -130 < difblt))) or
            len(labellev) > idx+1 and idx-1 >= 0 and 'BAHT' not in labellev[idx-1] and 
            ('BAHT' in labellev[idx+1] or 'FBAHT' in labellev[idx+1] or 'FYEAR' in labellev[idx+1])):
          otextlev[-1] += text
          olabellev[-1] += label
        else:
          print('prev', oxposlev[-1], otextlev[-1], olabellev[-1], onpagelev[-1])
          print('curr', xpos, text, label, npage)
          print(obfbltxlev[-1])
          print(bltwdt - 15, difblt, bltwdt - 15 < difblt, aftbltwdt - 20)
    else:
      oxposlev.append(xpos)
      otextlev.append(text)
      olabellev.append(label)
      onpagelev.append(npage)
      obfbltxlev.append(tbltx)
  if pl:
    for npge, x_, text, labl in zip(onpagelev, oxposlev, otextlev, olabellev):
      print(npge, x_ if x_ != np.int16(9990) else '---', [(tx, lb) for tx, lb in zip(text, labl)])

  return oxposlev, otextlev, olabellev, onpagelev

In [54]:
 def get_entry(cleaned_entry):
  xposlev, textlev, labellev, npagelev = cleaned_entry
  iserrorlev = [False] * len(xposlev)
  loglev = [''] * len(xposlev)

  for text, label, idx in zip(textlev, labellev, range(len(textlev))):
    if 'BULLET' not in label and 'FBAHT' not in label and 'OUTPUT' not in label and 'PROJECT' not in label:
      if '(ชดเชยงบ' in ''.join(text):
        xposlev.pop(idx)
        textlev.pop(idx)
        labellev.pop(idx)
        npagelev.pop(idx)
      elif get_patern_of_bullet(text[0]):
        label[0] = 'BULLET'
      else:
        print('🚨 [1] is not \'BULLET\': ', text)

  # get level dict
  lev = []
  level_tolerance = 11 
  for i in range(len(xposlev)):
    if lev == []:
      lev.append({'xPos': [xposlev[i]], 'index': [i]})
    else:
      diff = abs(np.subtract([np.mean(lv['xPos']) for lv in lev], xposlev[i]))
      if np.amin(diff) > level_tolerance:
        lev.append({'xPos': [xposlev[i]], 'index': [i]})
      else:
        idx = np.where(diff == np.amin(diff))[0][0]
        lev[idx]['xPos'].append(xposlev[i])
        lev[idx]['index'].append(i)

  # get level list of int
  level_dict = lev
  srtd_idx = sorted([(lv['index'], np.mean(lv['xPos'])) for lv in level_dict],
          key=lambda tup: tup[1])
  srtd_idx = [idx[0] for idx in srtd_idx]
  levs = []
  t_levs = len(srtd_idx)
  for idx in range(len(xposlev)):
    mask = np.array([idx in tst for tst in srtd_idx])
    n = np.arange(t_levs)[mask]
    levs.append(n[0])
 
  entry = []
  prjopt = {'Project': '', 'Output': ''}

  crdstack = []
  bltstack = []
  for pnum, xpos, level, text, label, idx in zip(npagelev, xposlev, levs, textlev, labellev, range(len(loglev))):
    if 'รายการบุคลากรภาครัฐ' in text:
      print('🚨 line skiped', pnum, level, text, label)
      iserrorlev[idx], loglev[idx] = True, 'LINE SKIPED'
      continue

    if 'วงเงินทั้งสิ้น' in text:
      continue

    if '(1) รายการไม่ผูกพัน' in ' '.join(text):
      continue

    context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NOTMATCH'])

    if 'OUTPUT' in label or 'PROJECT' in label:
      context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NOTMATCH' and txt != ':'])
      if 'OUTPUT' in label:
        prjopt['Output'] = context
      else:
        prjopt['Project'] = context
      continue
    elif 'FAMOUNT' in label or 'FBAHT' in label:
      amount = text[label.index('FAMOUNT')] if 'FAMOUNT' in label else '0'
      if entry:
        entry[-1]['is_obliged'] = 1
        if label.count('FYEAR') == 2:
          year = [txt for lbel, txt in zip(label, text) if lbel == 'FYEAR']
          fiscal_year = {i:amount for i in range(to_gregorian(year[0]), to_gregorian(year[1])+1)}
        elif label.count('FYEAR') == 1:
          fiscal_year = {to_gregorian(text[label.index('FYEAR')]): amount}
        else:
          print('🚨 no \'FYEAR\'', pnum, level, text, label)
          iserrorlev[idx], loglev[idx] = True, '\'FYEAR\' NOT FOUND'
          continue

        if isinstance(entry[-1]['fiscal_year'], int):
          entry[-1]['fiscal_year'] = fiscal_year
        elif isinstance(entry[-1]['fiscal_year'], dict):
          entry[-1]['fiscal_year'].update(fiscal_year)
        else:
          print('🚨 type not match', entry[-1]['fiscal_year'])
          iserrorlev[idx], loglev[idx] = True, 'TYPE NOT MATCH'
      else:
        print('🚨 entry is empty', pnum, level, text, label)
        iserrorlev[idx], loglev[idx] = True, 'ENTRY IS EMPTY'
        
      continue

    context = ' '.join([txt for lbel, txt in zip(label, text) if lbel == 'NOTMATCH'])
    amount = text[label.index('AMOUNT')] if 'AMOUNT' in label else '0'
    
    temp = {'context': context,
            'ref_line': text,
            'amount': amount,
            'coord_level': level,
            'page': pnum,
            'fiscal_year': int(curr_doc.fiscal_year),
            'is_obliged': 0,
            'DEBUG_LOG': ''}

    blt = 0
    bltlevel = -1
    clevel = level
    bltfound = 0
    if get_level_of_line(''.join(text)):
      bltfound = 1
      if 'BULLET' in label:
        blt = text[label.index('BULLET')]
      else:
        print('\'BULLET\' not found', text )
        iserrorlev[idx], loglev[idx] = True, '\'BULLET\' NOT FOUND'
        blt = text[0]
      if blt.count('('):
        clevel = 12
      bltlevel = get_level_of_line(''.join(text))
      temp['bullet_level'] = (bltlevel, blt)
    else:
      temp['DEBUG_LOG'] = 'NO BULLET FOUND'
      iserrorlev[idx], loglev[idx] = True, 'NO BULLET FOUND'

    if clevel == 12:
      temp['crdstack_level'] = 7
    else:
      db = crdstack[:]
      if crdstack:
        if crdstack[-1] < clevel:
          crdstack.append(clevel)
        elif crdstack[-1] > clevel:
          if not clevel in crdstack:
            crdstack[-1] = clevel
          while crdstack[-1] != clevel:
            crdstack.pop()
      else:
        crdstack.append(clevel)
      temp['crdstack_level'] = len(crdstack)

    if bltlevel != -1:
      if bltlevel == 12:
        temp['bltstack_level'] = 7
      else:
        db = bltstack[:]
        if bltstack:
          if bltstack[-1] < bltlevel:
            bltstack.append(bltlevel)
          elif bltstack[-1] > bltlevel:
            if not bltlevel in bltstack:
              bltstack[-1] = bltlevel
            while bltstack[-1] != bltlevel:
              bltstack.pop()
        else:
          bltstack.append(bltlevel)
        temp['bltstack_level'] = len(bltstack)
    else:
      temp['bltstack_level'] = len(bltstack)
      print('🚨 bulltet_lebel is -1: ', text, label)
    if temp['bltstack_level'] != temp['crdstack_level']:
      if bltfound:
        if temp['bltstack_level'] < temp['crdstack_level']:
          temp['level'] = temp['crdstack_level']
        else:
          temp['level'] = temp['bltstack_level']
      else:
        temp['level'] = temp['crdstack_level']

      iserrorlev[idx], loglev[idx] = True, 'LEVEL NOT MATCH'
      log = 'LEVEL NOT MATCH level: by x position {}, by pattern {}'.format(temp['crdstack_level'], temp['bltstack_level'])
      temp['DEBUG_LOG'] = temp['DEBUG_LOG'] + '/ ' + log if temp['DEBUG_LOG'] else log
    else:
      temp['level'] = temp['bltstack_level']

    if temp['DEBUG_LOG']:
      print(temp['DEBUG_LOG'])
      print('page: {}, [\'level\']: {}, xPos: {}, text: {}'.format(pnum, temp['level'], xpos, temp['ref_line']))
    
    entry.append(temp)
  
  semi_raw = []
  for pnum, xpos, level, text, label, iserror, log in zip(npagelev, xposlev, levs, textlev, labellev, iserrorlev, loglev):
    semi_raw.append({'text': text,
          'minxpos': xpos,
          'label': label,
          'page_number': pnum,
          'error_found': iserror,
          'log': log})

  return prjopt, entry, semi_raw

In [13]:
def get_categorys_lv(list_of_dict):
  CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG = [''] * 8
  for ite in list_of_dict:
    if ite['DEBUG_LOG']:
      DEBUG_LOG += ite['DEBUG_LOG']
    if ite['level'] == 1:
      CATEGORY_LV1 = ite['context']
    elif ite['level'] == 2:
      CATEGORY_LV2 = ite['context']
    elif ite['level'] == 3:
      CATEGORY_LV3 = ite['context']
    elif ite['level'] == 4:
      CATEGORY_LV4 = ite['context']
    elif ite['level'] == 5:
      CATEGORY_LV5 = ite['context']
    elif ite['level'] == 6:
      CATEGORY_LV6 = ite['context']
    elif ite['level'] == 7:
      ITEM_DESCRIPTION = ite['context']
  if DEBUG_LOG:
    DEBUG_LOG += '/ {}'.format([ite['ref_line'] for ite in list_of_dict])
  return CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG

In [14]:
def get_rows_of_budg_unit(running_id, budget_plan_details, bud_u):

  REF_DOC = curr_doc.ref_doc
  ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
  REF_PAGE_NO = -1
  DEBUG_LOG = ''
  MINISTRY = budget_plan_details['ministry']
  BUDGETARY_UNIT = budget_plan_details['budget_unit']
  CROSS_FUNC = budget_plan_details['is_cross_func']
  BUDGET_PLAN = budget_plan_details['budget_plan']
  OUTPUT = budget_plan_details['Output']
  PROJECT = budget_plan_details['Project']

  CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION = [''] * 7
  FISCAL_YEAR = curr_doc.fiscal_year
  AMOUNT = -1
  OBLIGED = 0

  stack = []
  rows = []
  for item in bud_u:
    if stack:
      if stack[-1]['level'] < item['level']:
        stack.append(item)
      elif stack[-1]['level'] > item['level']:
        rows.append([x for x in stack])
        stack.pop()
      elif stack[-1]['level'] == item['level']:
        rows.append([x for x in stack])
      if stack:
        stack.pop()
      stack.append(item)
    else:
      stack.append(item)
      
  rows.append([x for x in stack])

  rows_of_lists = []

  for row in rows[:]:
    if not row:
      continue
    itms = row[-1]
    REF_PAGE_NO = np.int16(itms['page'])
    OBLIGED = itms['is_obliged']
    CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4, CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, DEBUG_LOG = get_categorys_lv([x for x in row])

    if itms['is_obliged']:
      for fy, amt in itms['fiscal_year'].items():
        FISCAL_YEAR = fy
        AMOUNT = amt
        ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
        running_id += 1
        rows_of_lists.append([ITEM_ID, REF_DOC, np.int16(REF_PAGE_NO), MINISTRY, BUDGETARY_UNIT, np.bool_(CROSS_FUNC), BUDGET_PLAN, OUTPUT,
                              PROJECT, CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4,
                              CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION,
                              FISCAL_YEAR, AMOUNT, np.bool_(OBLIGED), DEBUG_LOG])
    else:
      FISCAL_YEAR = itms['fiscal_year']
      AMOUNT = itms['amount']
      ITEM_ID = '{}.{}'.format(REF_DOC, running_id)
      running_id += 1
      rows_of_lists.append([ITEM_ID, REF_DOC, np.int16(REF_PAGE_NO), MINISTRY, BUDGETARY_UNIT, np.bool_(CROSS_FUNC), BUDGET_PLAN, OUTPUT,
                            PROJECT, CATEGORY_LV1, CATEGORY_LV2, CATEGORY_LV3, CATEGORY_LV4,
                            CATEGORY_LV5, CATEGORY_LV6, ITEM_DESCRIPTION, FISCAL_YEAR, AMOUNT, np.bool_(OBLIGED), DEBUG_LOG])
  return running_id, rows_of_lists

In [26]:
def get_data_frame(doc):
  lemtestrange = doc.range
  for i, lrange in enumerate(lemtestrange[:]):
      print(i, lrange['budget_unit'], lrange['range'])

  pstart = doc.start
  listpage = []
  for lrange in lemtestrange:
      startp, endp = lrange['range']
      for ipage in range(startp, endp):
          listpage.append(doc.page(ipage + pstart))
          listpage[-1].pdfpage = ipage

  idtpages = []
  ihdpages = []
  tmphdict = {}
  semi_raw_entry = {doc.ref_doc :[]}
  Header = []
  Detail = []

  bgplan = ''
  lemrange = lemtestrange[:]

  for ipage in tqdm(listpage):
      text = '\n'.join(ipage.get_text_lines())
      headerw = ['หน่วย : ล้านบาท', 'หน่วยเล้านบาท', 'วัตถุประสงค์', 'หน่วย:ล้านบาท', 'หน่วยนับ']
      if not [hw for hw in headerw if hw in text]:
          # detail
          if ihdpages:
            tbgplan = get_budget_plan(ihdpages)
            bgplan = tbgplan if tbgplan else bgplan

            budget_unit = ''
            ministry = ''
            while not budget_unit:
              if lemrange[0]['range'][0] <= ihdpages[0].pdfpage < lemrange[0]['range'][1]:
                ministry = lemrange[0]['ministry']
                budget_unit = lemrange[0]['budget_unit']
              else:
                lemrange.pop(0)
            
            tmphdict = {'ministry': ministry, 'budget_unit': budget_unit,
                        'budget_plan': bgplan, 'ref_page': [i.pdfpage for i in ihdpages],
                        'is_cross_func': bgplan.startswith('แผนงานบูรณาการ')}
            ihdpages = []
          idtpages.append(ipage)
      else:
          # header
          if idtpages:
            prjopt, entry, semi_raw = get_entry(clean_entry(idtpages))
            semi_raw_entry[doc.ref_doc].append(semi_raw)
            Detail.append(entry)
            tmphdict.update(prjopt)
            Header.append(tmphdict)
            idtpages = []
          ihdpages.append(ipage)
  if idtpages:
    prjopt, entry, semi_raw = get_entry(clean_entry(idtpages))
    semi_raw_entry[doc.ref_doc].append(semi_raw)
    Detail.append(entry)
    tmphdict.update(prjopt)
    Header.append(tmphdict)
    
  df = pd.DataFrame(columns=cols)
  running_id = 0
  for header, detail in zip(Header, Detail):
    running_id, rows = get_rows_of_budg_unit(running_id, header, detail)
    new_row = pd.DataFrame(rows, columns=cols)
    df = df.append(new_row, ignore_index=True)

  print('entries contain error'.upper())
  ii = 0
  tempe = {}
  for buplan, header in tqdm(zip(semi_raw_entry[curr_doc.ref_doc], Header)):
    tempd = {}
    tempa = []
    curr_page = 0
    for entry in buplan:
      if entry['error_found']:
        print(entry)
      if curr_page and curr_page == entry['page_number']:
        tempa.append({
            'minxpos': entry['minxpos'],
            'error_found': entry['error_found'],
            'log': entry['log'],
            'block': [{'text': txt, 'label': lbel} for txt, lbel in zip(entry['text'], entry['label'])]
            })
      elif curr_page != entry['page_number']:
        if curr_page != 0:
          tempd.update({curr_page: tempa})
          tempa = []
        curr_page = entry['page_number']
    tempd.update({curr_page: tempa})
    header.update({'page': tempd})
    tempe.update({ii: header})
    ii += 1

  semirawentry = {curr_doc.ref_doc: tempe}

  with open("{}-raw-entry.json".format(curr_doc.ref_doc.replace('.', '-')), "w", encoding="utf8") as file:
    json.dump(semirawentry, file, ensure_ascii=False, cls=NpEncoder)

  df.to_csv('{}.csv'.format(curr_doc.ref_doc.replace('.', '-')))
  return df

# implementation

In [16]:
%%time
tbg = ThaiBudget('thaiBudget2022.json')

CPU times: user 9.47 s, sys: 5.45 s, total: 14.9 s
Wall time: 17 s


In [61]:
tbg.docs

['2022.3.1',
 '2022.3.2',
 '2022.3.3(1)',
 '2022.3.3(2)',
 '2022.3.3(3)',
 '2022.3.3(4)',
 '2022.3.3(5)',
 '2022.3.4',
 '2022.3.5',
 '2022.3.6',
 '2022.3.7',
 '2022.3.8',
 '2022.3.9',
 '2022.3.10',
 '2022.3.11',
 '2022.3.12',
 '2022.3.13(1)',
 '2022.3.13(2)',
 '2022.3.14',
 '2022.3.15',
 '2022.3.16(1)',
 '2022.3.16(2)',
 '2022.3.16(3)',
 '2022.3.17']

In [60]:
curr_doc = tbg.get_doc('2022.3.3(2)')
df = get_data_frame(curr_doc)

0 มหาวิทยาลัยราชภัฏธนบุรี (7, 19)
1 มหาวิทยาลัยราชภัฏนครปฐม (27, 43)
2 มหาวิทยาลัยราชภัฏนครราชสีมา (51, 66)
3 มหาวิทยาลัยราชภัฏนครศรีธรรมราช (73, 88)
4 มหาวิทยาลัยราชภัฏนครสวรรค์ (95, 107)
5 มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา (115, 130)
6 มหาวิทยาลัยราชภัฏบุรีรัมย์ (138, 156)
7 มหาวิทยาลัยราชภัฏพระนคร (162, 177)
8 มหาวิทยาลัยราชภัฏพระนครศรีอยุธยา (186, 198)
9 มหาวิทยาลัยราชภัฏพิบูลสงคราม (205, 222)
10 มหาวิทยาลัยราชภัฏเพชรบุรี (229, 247)
11 มหาวิทยาลัยราชภัฏเพชรบูรณ์ (254, 264)
12 มหาวิทยาลัยราชภัฏภูเก็ต (271, 287)
13 มหาวิทยาลัยราชภัฏมหาสารคาม (295, 306)
14 มหาวิทยาลัยราชภัฏยะลา (315, 341)
15 มหาวิทยาลัยราชภัฏร้อยเอ็ด (349, 366)
16 มหาวิทยาลัยราชภัฏราชนครินทร์ (373, 388)
17 มหาวิทยาลัยราชภัฏรำไพพรรณี (395, 413)
18 มหาวิทยาลัยราชภัฏลำปาง (421, 436)
19 มหาวิทยาลัยราชภัฏเลย (443, 459)
20 มหาวิทยาลัยราชภัฏวไลยอลงกรณ์ ในพระบรมราชูปถัมภ์ จังหวัดปทุมธานี (468, 483)
21 มหาวิทยาลัยราชภัฏศรีสะเกษ (491, 506)


LEVEL NOT MATCH level: by x position 3, by pattern 2
page: 77, ['level']: 3, xPos: 249, text: ['1)', 'ค่าใช้จ่ายดำเนินงาน', '8,327,500', 'บาท']
LEVEL NOT MATCH level: by x position 3, by pattern 2
page: 77, ['level']: 3, xPos: 248, text: ['2)', 'เงินอุดหนุนโครงการตามพระราชดำริ', '1,000,000', 'บาท']
🚨 [1] is not 'BULLET':  ['ค่าใช้จ่ายดำเนินงาน', '6,817,000', 'บาท']
🚨 bulltet_lebel is -1:  ['ค่าใช้จ่ายดำเนินงาน', '6,817,000', 'บาท'] ['NOTMATCH', 'AMOUNT', 'BAHT']
NO BULLET FOUND/ LEVEL NOT MATCH level: by x position 3, by pattern 2
page: 82, ['level']: 3, xPos: 211, text: ['ค่าใช้จ่ายดำเนินงาน', '6,817,000', 'บาท']
LEVEL NOT MATCH level: by x position 2, by pattern 3
page: 124, ['level']: 3, xPos: 180, text: ['1)', 'เงินอุดหนุนค่าใช้จ่ายโครงการอนุรักษ์พันธุกรรมพืชอันเนื่องมาจากพระราชดำริฯ', '3,843,000', 'บาท']
LEVEL NOT MATCH level: by x position 2, by pattern 3
page: 127, ['level']: 3, xPos: 180, text: ['1)', 'เงินอุดหนุนค่าใช้จ่ายโครงการยุทธศาสตร์มหาวิทยาลัยราชภัฏเพื่อการพัฒนาท้องถิ่น

{'text': ['1)', 'ค่าใช้จ่ายดำเนินงาน', '8,327,500', 'บาท'], 'minxpos': 249, 'label': ['BULLET', 'NOTMATCH', 'AMOUNT', 'BAHT'], 'page_number': 77, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['2)', 'เงินอุดหนุนโครงการตามพระราชดำริ', '1,000,000', 'บาท'], 'minxpos': 248, 'label': ['BULLET', 'NOTMATCH', 'AMOUNT', 'BAHT'], 'page_number': 77, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['ค่าใช้จ่ายดำเนินงาน', '6,817,000', 'บาท'], 'minxpos': 211, 'label': ['NOTMATCH', 'AMOUNT', 'BAHT'], 'page_number': 82, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['1)', 'เงินอุดหนุนค่าใช้จ่ายโครงการอนุรักษ์พันธุกรรมพืชอันเนื่องมาจากพระราชดำริฯ', '3,843,000', 'บาท'], 'minxpos': 180, 'label': ['BULLET', 'NOTMATCH', 'AMOUNT', 'BAHT'], 'page_number': 124, 'error_found': True, 'log': 'LEVEL NOT MATCH'}
{'text': ['1)', 'เงินอุดหนุนค่าใช้จ่ายโครงการยุทธศาสตร์มหาวิทยาลัยราชภัฏเพื่อการพัฒนาท้องถิ่น', '35,533,500', 'บาท'], 'minxpos': 180, 'label': ['BULLET', 'NOTMATCH', 'AMOUNT', '

In [28]:
a = clean_entry([curr_doc.page(i, True) for i in [97]], pl=1)
a = get_entry(a)

97 201 [('โครงการ', 'PROJECT'), (':', 'NOTMATCH'), ('โครงการสนับสนุนการจัดตั้งห้องเรียนวิทยาศาสตร์', 'NOTMATCH'), ('ในโรงเรียน', 'NOTMATCH'), ('โดยการกำกับดูแลของมหาวิทยาลัย', 'NOTMATCH'), ('(ระยะที่', 'NOTMATCH'), ('2)', 'NOTMATCH'), ('257,789,600', 'AMOUNT'), ('บาท', 'BAHT')]
97 204 [('1.', 'BULLET'), ('งบเงินอุดหนุน', 'NOTMATCH'), ('257,789,600', 'AMOUNT'), ('บาท', 'BAHT')]
97 226 [('1.1', 'BULLET'), ('เงินอุดหนุนทั่วไป', 'NOTMATCH'), ('257,789,600', 'AMOUNT'), ('บาท', 'BAHT')]
97 250 [('1)', 'BULLET'), ('เงินอุดหนุนสำหรับสนับสนุนห้องเรียนวิทยาศาสตร์และเทคโนโลยี', 'NOTMATCH'), ('(ระยะที่', 'NOTMATCH'), ('2)', 'NOTMATCH'), ('257,789,600', 'AMOUNT'), ('บาท', 'BAHT')]


In [62]:
df.loc[df['DEBUG_LOG'] == '']

,ITEM_ID,REF_DOC,REF_PAGE_NO,MINISTRY,BUDGETARY_UNIT,CROSS_FUNC?,BUDGET_PLAN,OUTPUT,PROJECT,CATEGORY_LV1,CATEGORY_LV2,CATEGORY_LV3,CATEGORY_LV4,CATEGORY_LV5,CATEGORY_LV6,ITEM_DESCRIPTION,FISCAL_YEAR,AMOUNT,OBLIGED?,DEBUG_LOG
0,2022.3.3(2).0,2022.3.3(2),8,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏธนบุรี,False,แผนงานบุคลากรภาครัฐ,,,งบบุคลากร,เงินเดือนและค่าจ้างประจำ,เงินเดือน,,,,,2022,"30,387,500",False,
1,2022.3.3(2).1,2022.3.3(2),8,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏธนบุรี,False,แผนงานบุคลากรภาครัฐ,,,งบบุคลากร,เงินเดือนและค่าจ้างประจำ,ค่าจ้างประจำ,,,,,2022,"3,447,000",False,
2,2022.3.3(2).2,2022.3.3(2),8,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏธนบุรี,False,แผนงานบุคลากรภาครัฐ,,,งบบุคลากร,ค่าจ้างชั่วคราว,,,,,,2022,"2,256,700",False,
3,2022.3.3(2).3,2022.3.3(2),8,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏธนบุรี,False,แผนงานบุคลากรภาครัฐ,,,งบบุคลากร,ค่าตอบแทนพนักงานราชการ,,,,,,2022,"3,432,400",False,
4,2022.3.3(2).4,2022.3.3(2),8,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏธนบุรี,False,แผนงานบุคลากรภาครัฐ,,,งบดำเนินงาน,ค่าตอบแทน ใช้สอยและวัสดุ,,,,,ค่าตอบแทนพิเศษข้าราชการที่ได้รับเงินเดือนเต็มขั้น,2022,"22,100",False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,2022.3.3(2).934,2022.3.3(2),505,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏศรีสะเกษ,False,แผนงานยุทธศาสตร์สร้างความเสมอภาคทางการศึกษา,,โครงการสนับสนุนค่าใช้จ่ายในการจัดการศึกษาตั้งแ...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,ค่าใช้จ่ายในการจัดการศึกษาขั้นพื้นฐาน,ค่าจัดการเรียนการสอน,,,,2022,"625,600",False,
935,2022.3.3(2).935,2022.3.3(2),505,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏศรีสะเกษ,False,แผนงานยุทธศาสตร์สร้างความเสมอภาคทางการศึกษา,,โครงการสนับสนุนค่าใช้จ่ายในการจัดการศึกษาตั้งแ...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,ค่าใช้จ่ายในการจัดการศึกษาขั้นพื้นฐาน,ค่าหนังสือเรียน,,,,2022,"128,500",False,
936,2022.3.3(2).936,2022.3.3(2),505,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏศรีสะเกษ,False,แผนงานยุทธศาสตร์สร้างความเสมอภาคทางการศึกษา,,โครงการสนับสนุนค่าใช้จ่ายในการจัดการศึกษาตั้งแ...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,ค่าใช้จ่ายในการจัดการศึกษาขั้นพื้นฐาน,ค่าอุปกรณ์การเรียน,,,,2022,"91,200",False,
937,2022.3.3(2).937,2022.3.3(2),505,กระทรวงการอุดมศึกษา วิทยาศาสตร์ วิจัยและนวัตกร...,มหาวิทยาลัยราชภัฏศรีสะเกษ,False,แผนงานยุทธศาสตร์สร้างความเสมอภาคทางการศึกษา,,โครงการสนับสนุนค่าใช้จ่ายในการจัดการศึกษาตั้งแ...,งบเงินอุดหนุน,เงินอุดหนุนทั่วไป,ค่าใช้จ่ายในการจัดการศึกษาขั้นพื้นฐาน,ค่าเครื่องแบบนักเรียน,,,,2022,"125,100",False,


In [63]:
df.loc[df['DEBUG_LOG'] != ''].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 161 to 847
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ITEM_ID           8 non-null      object
 1   REF_DOC           8 non-null      object
 2   REF_PAGE_NO       8 non-null      object
 3   MINISTRY          8 non-null      object
 4   BUDGETARY_UNIT    8 non-null      object
 5   CROSS_FUNC?       8 non-null      object
 6   BUDGET_PLAN       8 non-null      object
 7   OUTPUT            8 non-null      object
 8   PROJECT           8 non-null      object
 9   CATEGORY_LV1      8 non-null      object
 10  CATEGORY_LV2      8 non-null      object
 11  CATEGORY_LV3      8 non-null      object
 12  CATEGORY_LV4      8 non-null      object
 13  CATEGORY_LV5      8 non-null      object
 14  CATEGORY_LV6      8 non-null      object
 15  ITEM_DESCRIPTION  8 non-null      object
 16  FISCAL_YEAR       8 non-null      object
 17  AMOUNT          

In [21]:
df.to_csv('{}.csv'.format(curr_doc.ref_doc.replace('.', '-')))

In [22]:
curr_doc.page(332, True).whpage

array([[ 114, 1478],
       [ 100, 1848]])

In [23]:
xposl, textl = curr_doc.page(332, True).xpos_text_lines()
for xpos, line in zip(xposl, textl):
  print([(txt, x) for x, txt in zip(xpos.tolist(), line)])

[('332', [[167, 211], [100, 122]])]
[('7.2', [[120, 145], [175, 189]]), ('แผนงานพื้นฐานด้านการพัฒนาและเสริมสร้างศักยภาพทรัพยากรมนุษย์', [[164, 777], [162, 198]]), ('116,670,600', [[1207, 1313], [174, 192]]), ('บาท', [[1338, 1373], [176, 189]])]
[('7.2.1', [[217, 257], [222, 238]]), ('ผลผลิตที่', [[276, 354], [210, 239]]), ('1', [[366, 373], [223, 237]]), (':', [[383, 386], [227, 238]]), ('ผู้สำเร็จการศึกษาด้านสังคมศาสตร์', [[405, 698], [211, 244]]), ('58,947,100', [[1217, 1313], [222, 241]]), ('บาท', [[1338, 1373], [224, 237]])]
[('วัตถุประสงค์', [[248, 357], [262, 295]]), (':', [[366, 367], [276, 287]])]
[('จัดการศึกษาเพื่อผลิตบัณฑิตด้านสังคมศาสตร์ตอบสนองความต้องการของผู้เรียนและท้องถิ่น', [[302, 1076], [311, 342]])]
[('7.2.1.1', [[217, 274], [368, 383]]), ('งบประมาณรายจ่าย', [[293, 457], [362, 383]]), ('จำแนกตามกิจกรรม-งบรายจ่าย', [[467, 730], [362, 383]])]
[('หน่วย:ล้านบาท', [[1351, 1477], [417, 442]])]
[('กิจกรรม', [[393, 461], [491, 513]]), ('งบรายจ่าย', [[1067, 1154], [467, 489]]